# Import Library

# Option Pricing with Black-Scholes Model

**Assumption from Black-Scholes Model:**
* No dividends are paid out during the life of the option
* Markets are random 
* No transaction costs when buying the option
* Risk-Free Rate and volatility of underlying asset are known and constant
* Returns of underlying asset are normally distributed
* Option is European

In [1]:
class BS_Option_Pricing:

    def __init__(self, spot, strike, maturity, rate, vol, dividend: float = None):
        self.spot = spot
        self.strike = strike
        self.maturity = maturity
        self.rate = rate
        self.vol = vol
        self.div = dividend

    def d1(self, add_vol: int = None):
        vol = add_vol if add_vol else self.vol
        return (np.log((np.exp(-self.div * self.maturity) if self.div else 1) * self.spot / self.strike)
                + (self.rate + vol ** 2 / 2) * self.maturity) / (vol * np.sqrt(self.maturity))

    def d2(self, add_vol: int = None):
        vol = add_vol if add_vol else self.vol
        return self.d1() - vol * np.sqrt(self.maturity)

    def call_price(self, add_vol: int = None):
        return self.spot * norm.cdf(self.d1(add_vol)) * (np.exp(- self.div * self.maturity) if self.div else 1) - \
               self.strike * np.exp(-self.rate * self.maturity) * norm.cdf(self.d2(add_vol))

    def put_price(self, add_vol: int = None):
        return self.strike * np.exp(-self.rate * self.maturity) * norm.cdf(-self.d2(add_vol)) - self.spot * norm.cdf(
            -self.d1(add_vol) * (np.exp(- self.div * self.maturity) if self.div else 1))

    def call_implied_vol(self, target_price: int, tol: int = .001):
        sigma = tol
        while sigma < 1:
            temp_price = self.call_price(sigma)
            if abs(target_price - temp_price) < tol:
                return sigma
            else:
                sigma += tol
        return "Error when solving!"

    def put_implied_vol(self, target_price: int, tol: int = .001):
        sigma = tol
        while sigma < 1:
            temp_price = self.put_price(sigma)
            if abs(target_price - temp_price) < tol:
                return sigma
            else:
                sigma += tol
        return "Error when solving!"


# Greek from Option Study

In [2]:
class Greek(BS_Option_Pricing):

    def __init__(self, spot, strike, maturity, rate, vol, dividend: float = None):
        BS_Option_Pricing.__init__(self, spot, strike, maturity, rate, vol, dividend)

    def call_delta(self):
        return norm.cdf(self.d1())

    def put_delta(self):
        return -norm.cdf(-self.d1())

    def call_gamma(self):
        return norm.pdf(self.d1()) / (self.spot * self.vol * sqrt(self.maturity))

    def put_gamma(self):
        return self.call_gamma()

    def call_vega(self):
        return self.spot * np.sqrt(self.maturity) * norm.pdf(self.d1())

    def put_vega(self):
        return self.call_vega()

    def call_theta(self):
        return - ((self.spot * norm.pdf(self.d1()) * self.vol) / (
                    2 * self.maturity)) - self.rate * self.strike * np.exp(
            - self.rate * self.maturity) * norm.cdf(self.d2())

    def put_theta(self):
        return - ((self.spot * norm.pdf(self.d1()) * self.vol) / (
                    2 * self.maturity)) + self.rate * self.strike * np.exp(
            - self.rate * self.maturity) * norm.cdf(self.d2())

    def call_rho(self):
        return self.strike * self.maturity * np.exp(- self.rate * self.maturity) * norm.cdf(self.d2())

    def put_rho(self):
        return self.strike * self.maturity * np.exp(- self.rate * self.maturity) * norm.cdf(self.d2())

    "ADD EXOTIC GREEKS?"

    def get_details(self, kind: str):
        full_info = {
            "Option Price": self.call_price() if kind == "call" else self.put_price(),
            "Option Delta": self.call_delta() if kind == "call" else self.put_delta(),
            "Option Gamma": self.call_gamma() if kind == "call" else self.put_gamma(),
            "Option Vega": self.call_vega() if kind == "call" else self.put_vega(),
            "Option Theta": self.call_theta() if kind == "call" else self.put_theta(),
            "Option Rho": self.call_rho() if kind == "call" else self.put_rho()
        }
        return full_info


# Main strategies payoff overview

In [3]:
class Strategies_payoffs(BS_Option_Pricing):

    def __init__(self, spot, strike, maturity, rate, vol, dividend: float = None):
        BS_Option_Pricing.__init__(self, spot, strike, maturity, rate, vol, dividend)


    def simple_option(self, option_type: str = "put", kind: str = "short"):
        option_price = BS_Option_Pricing.put_price(self) if option_type == "put" else BS_Option_Pricing.call_price(self)
        payoff = np.max(self.spot - self.strike, -option_price) if option_type == "call" else np.max(self.strike - self.spot, - option_price)
        return payoff if kind == "long" else - payoff

    def bull_spread(self):
        return self.simple_option("call", "long") + self.simple_option("call", "short")

    def straddle(self, kind: str = "short"):
        put = BS_Option_Pricing.call_price(self)
        call = BS_Option_Pricing.put_price(self)
        abs_payoff = put + call
        return abs_payoff if kind == "long" else - abs_payoff

    def strangle(self, strike2: float, kind: str = "short"):
        if not (self.strike < strike2):
            print("Second Asset should have strike price lower than first one!")
            pass
        else:
            put = BS_Option_Pricing.put_price(self)
            call = BS_Option_Pricing(self.spot, strike2, self.maturity, self.rate, self.vol).call_price()
            abs_payoff = put + call
            return abs_payoff if kind == "long" else - abs_payoff

    def butterfly(self, strike2: float, strike3: float, kind: str = "short"):
        if not (self.strike < strike2 < strike3):
            print("We need Strike1 < Strike2 < Strike3")
        else:
            option1 = BS_Option_Pricing.call_price(self) if kind == "long" else BS_Option_Pricing.put_price(self)
            option2 = BS_Option_Pricing(self.spot, strike2, self.maturity, self.rate, self.vol).call_price() if kind == "long" else \
                      BS_Option_Pricing(self.spot, strike2, self.maturity, self.rate, self.vol).put_price()
            option3 = BS_Option_Pricing(self.spot, strike3, self.maturity, self.rate,self.vol).call_price() if kind == "long" else \
                      BS_Option_Pricing(self.spot, strike3, self.maturity, self.rate, self.vol).put_price()
            abs_payoff = option1 - 2 * option2 + option3
            return abs_payoff if kind == "long" else - abs_payoff

    def condor(self, strike2: float, strike3: float, strike4: float, kind: str = "short"):
        if not (self.strike < strike2 < strike3 < strike4):
            print("We need Strike1 < Strike2 < Strike3 < Strike4")
        else:
            option1 = BS_Option_Pricing.put_price(self) if kind == "long" else BS_Option_Pricing.call_price(self)
            option2 = BS_Option_Pricing(self.spot, strike2, self.maturity, self.rate, self.vol).put_price() if kind == "long" else \
                      BS_Option_Pricing(self.spot, strike2, self.maturity, self.rate, self.vol).call_price()
            option3 = BS_Option_Pricing(self.spot, strike3, self.maturity, self.rate,self.vol).call_price() if kind == "long" else \
                      BS_Option_Pricing(self.spot, strike3, self.maturity, self.rate, self.vol).put_price()
            option4 = BS_Option_Pricing(self.spot, strike4, self.maturity, self.rate,self.vol).call_price() if kind == "long" else \
                      BS_Option_Pricing(self.spot, strike4, self.maturity, self.rate, self.vol).put_price()
            abs_payoff = - option1 + option2 + option3 - option4
            return abs_payoff if kind == "long" else - abs_payoff
